# 75ML30D
![63%](https://progress-bar.dev/63?title=progress&width=100)

#### Description:
- Solving 75 Medium Leetcode problems in 30 days.<br>
- Languange of choice: `Python`<br>

#### Timelines:
- Start Date: 9th Jan 2024
- Challenge End Date: 8th Feb 2024

In [1]:
from typing import List
import pandas as pd
import numpy as np
import sqlite3
import re
import io
import math
import collections
import itertools
import functools
import random
import string
import tqdm
import bisect

conn = sqlite3.connect(":memory:")

def regexp(expr, item):
    reg = re.compile(expr)
    return reg.search(item) is not None

def read_lc_df(s: str) -> pd.DataFrame:
    temp = pd.read_csv(io.StringIO(s), sep="|", skiprows=2)
    temp = temp.iloc[1:-1, 1:-1]
    temp.columns = temp.columns.map(str.strip)
    temp = temp.map(lambda x: x if type(x) != str else None if x.strip() == 'null' else x.strip())
    return temp
    
conn.create_function("REGEXP", 2, regexp)

#### Helper for Binary tree problems

In [2]:
class BinaryTreeNode:
    def __init__(self, val=0, left=None, right=None):
        self.val = val
        self.left = left
        self.right = right

    def to_list(self):
        to_visit = [self]
        visited = []
        while len(to_visit) > 0:
            curr = to_visit.pop(0)
            if curr:
                to_visit.append(curr.left)
                to_visit.append(curr.right)
                visited.append(curr.val)
            else:
                visited.append(curr)
        return visited

    def __str__(self):
        return str(self.val)

    def from_array(nums: List[int]):
        '''Create a Tree from a list of nums. Returns the root node.'''
        if len(nums) == 0:
            return None
        elif len(nums) == 1:
            return BinaryTreeNode(nums[0])
        else:
            forest = [BinaryTreeNode(nums[0])]
            parent_idx = -1
            for i in range(1, len(nums)):

                curr = None
                if nums[i] is not None:
                    curr = BinaryTreeNode(nums[i])
                    forest.append(curr)
                
                if i % 2 == 1:
                    parent_idx += 1
                    forest[parent_idx].left = curr
                else:
                    forest[parent_idx].right = curr

        return forest[0]

temp = BinaryTreeNode.from_array([1,2,3,4,5,6,None,None,None,7,8])
temp.to_list()

[1, 2, 3, 4, 5, 6, None, None, None, 7, 8, None, None, None, None, None, None]

#### Helper for Singly Linked lists

In [3]:
class ListNode:
    def __init__(self, val=0, next=None):
        self.val = val
        self.next = next

    def __str__(self):
        return str(self.val)

    def to_singly_linked_list(nums: list[int]):
        root = prev = None
        for n in nums:
            curr = ListNode(n)
            # Init once
            if not root:
                root = curr
            if prev:
                prev.next = curr
            prev = curr
            
        return root

    def to_list(self) -> list[int]:
        result = []
        curr = self
        while curr:
            result.append(curr.val)
            curr = curr.next
        return result

# 1. 
Date: 09th Jan 2024<br>
Medium Leetcode: minimum-path-sum<br>
Solution: https://leetcode.com/problems/minimum-path-sum/submissions/1141069624

In [4]:
class Solution:
    def minPathSum(self, grid: List[List[int]]) -> int:
        # Compute once
        height = len(grid)
        width = len(grid[0])

        # Start at the bottom right
        x, y = height - 1, width - 1

        while x >= 0:
            while y >= 0:
                
                add = min(
                    grid[x][y + 1] if y + 1 < width else math.inf,
                    grid[x + 1][y] if x + 1 < height else math.inf
                )
                
                if add != math.inf:
                    grid[x][y] += add
                    
                # Decrement y (go left)
                y -= 1

            # Decrement x (go up)
            x -= 1

            # Reset y
            y = width - 1

        return grid[0][0]

# Test the solution
assert Solution().minPathSum([[1,3,1],[1,5,1],[4,2,1]]) == 7
assert Solution().minPathSum([[1,2,3],[4,5,6]]) == 12

# 2. 
Date: 10th Jan 2024<br>
Medium Leetcode: amount-of-time-for-binary-tree-to-be-infected<br>
Solution: https://leetcode.com/problems/amount-of-time-for-binary-tree-to-be-infected/submissions/1141991104

In [5]:
class Solution:
    def amountOfTime(self, root: BinaryTreeNode, start: int) -> int:
        matrix = dict()
        def getAdjacencyMatrix(curr, parent=None):
            if curr:
                vertices = matrix.get(curr, [])

                if parent:
                    vertices.append(parent.val)
                if curr.right:
                    vertices.append(curr.right.val)
                if curr.left:
                    vertices.append(curr.left.val)

                matrix[curr.val] = vertices
    
                # Recurse
                getAdjacencyMatrix(curr.left, curr)
                getAdjacencyMatrix(curr.right, curr)
            

        getAdjacencyMatrix(root)

        # Do a BFS until all nodes are exhausted
        minute = 0
        infected = set({start})
        neighbours = matrix[start]

        while neighbours:
            next_neighbours = []
            for node in neighbours:
                if node not in infected:
                    infected.add(node)
                    next_neighbours.extend(list(filter(lambda x: x not in infected, matrix[node])))

            neighbours = next_neighbours
            minute += 1
                
        return minute
        
# Test the solution
assert Solution().amountOfTime(BinaryTreeNode.from_array([1,5,3,None,4,10,6,9,2]), 3) == 4
assert Solution().amountOfTime(BinaryTreeNode.from_array([1]), 1) == 0

# 3. 
Date: 10th Jan 2024<br>
Medium Leetcode: dota2-senate<br>
Solution: https://leetcode.com/problems/dota2-senate/submissions/1142503920/

In [6]:
class Solution:
    def predictPartyVictory(self, senate: str) -> str:

        while senate:

            result = []
            count = 0
            for ch in senate:
                if count == 0:
                    result.append(ch)
                    count = 1
                elif ch != result[-1]:
                    count -= 1
                else:
                    result.append(ch)
                    count += 1
            
            new_result = []
            for ch in result:
                if count > 0 and ch != result[-1]:
                    count -= 1
                else:
                    new_result.append(ch)
    
            senate = "".join(new_result)

            if count == len(senate):
                return "Radiant" if senate[0] == "R" else "Dire"

# Test the solution
assert Solution().predictPartyVictory("RDRDDDRDDRDRDDRRRRRD") == "Dire"
assert Solution().predictPartyVictory("RDRDDRD") == "Dire"
assert Solution().predictPartyVictory("RRDDD") == "Radiant"

# 4. 
Date: 11th Jan 2024<br>
Medium Leetcode: populating-next-right-pointers-in-each-node<br>
Solution: https://leetcode.com/problems/populating-next-right-pointers-in-each-node/submissions/1142856094/

In [7]:
class Solution:
    def connect(self, root):
        'This takes O(log(n)) space and O(1) complexity, q asked for O(1) space' 

        def backtrack(curr: Node, level: int = 0, right_most: dict = {}):
            if not curr:
                return 

            if level in right_most:
                right_most[level].next = curr

            right_most[level] = curr
            backtrack(curr.left, level + 1, right_most)
            backtrack(curr.right, level + 1, right_most)

        backtrack(root)
        return root

# 5. 
Date: 11th Jan 2024<br>
Medium Leetcode: maximum-difference-between-node-and-ancestor<br>
Solution: https://leetcode.com/problems/maximum-difference-between-node-and-ancestor/submissions/1142882452

In [8]:
class Solution:
    def maxAncestorDiff(self, root: BinaryTreeNode) -> int:
        
        def backtrack(curr):
            
            if not curr:
                return None, None, None

            lmin, lmax, ldiff = backtrack(curr.left) 
            rmin, rmax, rdiff = backtrack(curr.right)
            currMin = currMax = curr.val
            maxDiff = -math.inf

            if curr.left:
                currMin = min(currMin, lmin)
                currMax = max(currMax, lmax)
                maxDiff = max(
                    maxDiff,
                    abs(curr.val - lmin),
                    abs(curr.val - lmax),
                    ldiff
                )

            if curr.right:
                currMin = min(currMin, rmin)
                currMax = max(currMax, rmax)
                maxDiff = max(
                    maxDiff,
                    abs(curr.val - rmin),
                    abs(curr.val - rmax),
                    rdiff
                )

            return currMin, currMax, maxDiff

        return backtrack(root)[2]

# Test ths solution
assert Solution().maxAncestorDiff(BinaryTreeNode.from_array([8,3,10,1,6,None,14,None,None,4,7,13])) == 7
assert Solution().maxAncestorDiff(BinaryTreeNode.from_array([8,3,10,1,6,None,14,None,None,50,7,13])) == 47
assert Solution().maxAncestorDiff(BinaryTreeNode.from_array([1,None,2,None,0,3])) == 3

# 6. 
Date: 11th Jan 2024<br>
Medium Leetcode: delete-the-middle-node-of-a-linked-list<br>
Solution: https://leetcode.com/problems/delete-the-middle-node-of-a-linked-list/submissions/1142953098

In [9]:
class Solution:
    def deleteMiddle(self, head: ListNode) -> ListNode:
        'Better approach after seeing the solution is to use the fast pointer / slow pointer approach'
        n, curr = 0, head  
        while curr:
            curr = curr.next
            n += 1

        mid = n // 2

        if mid == 0:
            return None

        prev, curr = None, head
        while mid > 0:
            prev, curr = curr, curr.next
            mid -= 1

        prev.next = curr.next
        curr.next = None
        return head

# Test the solution
assert Solution().deleteMiddle(ListNode.to_singly_linked_list([1])) is None
assert Solution().deleteMiddle(ListNode.to_singly_linked_list([1, 2])).to_list() == [1]
assert Solution().deleteMiddle(ListNode.to_singly_linked_list([1, 2, 3])).to_list() == [1, 3]
assert Solution().deleteMiddle(ListNode.to_singly_linked_list([1, 2, 3, 4])).to_list() == [1, 2, 4]

# 7. 
Date: 12th Jan 2024<br>
Medium Leetcode: find-minimum-in-rotated-sorted-array<br>
Solution: https://leetcode.com/problems/find-minimum-in-rotated-sorted-array/submissions/1144294358/

In [10]:
class Solution:
    def findMin(self, nums: List[int]) -> int:
        """
        Objective is to find i such that nums[i] < nums[i - 1]
        """

        # O(1) time complexity
        n = len(nums)

        # Edge case - 1
        if n == 1:
            return nums[0]

        # Edge case - 2
        elif nums[0] < nums[-1]:
            return nums[0]
        
        else:
            start, end = 1, n - 1
            while start <= end:
                mid = (start + end) // 2

                if nums[mid] < nums[mid - 1]:
                    return nums[mid]
                
                elif nums[mid] > nums[0]:
                    start = mid + 1
                
                else:
                    end = mid - 1
            return -1

# Test the solution
assert Solution().findMin([4,5,6,7,0,1,2]) == 0
assert Solution().findMin([11,13,15,17]) == 11
assert Solution().findMin([2,3,5,6,7,1]) == 1
assert Solution().findMin([2,3,5,6,7,8,1]) == 1
assert Solution().findMin([6,7,8,1,2,3,4,5]) == 1

# 8. 
Date: 12th Jan 2024<br>
Medium Leetcode: count-and-say<br>
Solution: https://leetcode.com/problems/count-and-say/submissions/1144310034/

In [11]:
class Solution:
    def countAndSay(self, n: int) -> str:
        def say(s: str):
            prev = None
            count = 0
            result = []
            for ch in s + " ":
                if prev != ch:
                    if prev:
                        result.append(f"{count}{prev}")
                    prev, count = ch, 1
                else:
                    count += 1
                    
            return "".join(result)

        curr = "1" 
        while n > 1:
            curr = say(curr)
            n -= 1

        return curr

# Test the solution
assert Solution().countAndSay(10) == "13211311123113112211"
assert Solution().countAndSay(4) == "1211"

# 9. 
Date: 13th Jan 2024<br>
Medium Leetcode: minimum-number-of-steps-to-make-two-strings-anagram<br>
Solution: https://leetcode.com/problems/minimum-number-of-steps-to-make-two-strings-anagram/submissions/1144703174

In [12]:
class Solution:
    def minSteps(self, s: str, t: str) -> int:
        currentCounter = collections.Counter(s)
        targetCounter = collections.Counter(t)
        
        count = 0
        for k, v in currentCounter.items():
            count += max(v - targetCounter.get(k, 0), 0)
        
        return count
    
# Test the solution
assert Solution().minSteps("leetcode", "practice") == 5
assert Solution().minSteps("anagram", "mangaar") == 0
assert Solution().minSteps("bab", "aba") == 1

# 10. 
Date: 13th Jan 2024<br>
Medium Leetcode: longest-common-subsequence<br>
Solution: https://leetcode.com/problems/longest-common-subsequence/submissions/1144816986

In [13]:
class Solution:
    def longestCommonSubsequence(self, text1: str, text2: str) -> int:
        
        n1, n2 = len(text1), len(text2)

        def iterate() -> int:
            prev = [0 for _ in range(n1 + 1)]
            for i in range(n2 - 1, -1, -1):
                curr = [0 for _ in range(n1 + 1)]
                for j in range(n1 - 1, -1, -1):
                    if text1[j] == text2[i]:
                        curr[j] = 1 + prev[j + 1]
                    else:
                        curr[j] = max(prev[j], curr[j + 1])
                prev = curr
                        
            return curr[0]

        @functools.cache
        def backtrack(i: int, j: int) -> int:
            if i == n1 or j == n2:
                return 0
            elif text1[i] == text2[j]:
                return 1 + backtrack(i + 1, j + 1)
            else:
                return max(backtrack(i, j + 1), backtrack(i + 1, j))
        
        return iterate()

# Testing the solution
assert Solution().longestCommonSubsequence("abcded", "acede") == 4
assert Solution().longestCommonSubsequence("", "acede") == 0

# 11. 
Date: 13th Jan 2024<br>
Medium Leetcode: edit-distance<br>
Solution: https://leetcode.com/problems/edit-distance/submissions/1145197901/

In [14]:
class Solution:
    def minDistance(self, word1: str, word2: str) -> int:
        """
        Remove / insert characters towards the end to match length
        Replace all characters with characters from word2
        Upper bound = max(n1, n2) - min(n1, n2) + n2

        # Optimized approach after watching 2 min of neetcode - watch tutorial
        """

        # Find the length of both the words
        n1, n2 = len(word1), len(word2) 

        @functools.cache
        def backtrack(i: int, j: int) -> int:
            if i >= n1 or j >= n2:
                return max(n1 - i, n2 - j)
            elif word1[i] == word2[j]:
                return backtrack(i + 1, j + 1)
            else:
                return 1 + min(
                    # Replace current char
                    backtrack(i + 1, j + 1),

                    # Remove current char
                    backtrack(i + 1, j),

                    # Insert char from target string
                    backtrack(i, j + 1)
                )
        
        answer = backtrack(0, 0)
        return answer

# Testing the solution
assert Solution().minDistance("horse", "ros") == 3
assert Solution().minDistance("intention", "execution") == 5
assert Solution().minDistance("mississippi", "missouri") == 6
assert Solution().minDistance("", "abcd") == 4
assert Solution().minDistance("abcd", "") == 4
assert Solution().minDistance("", "") == 0

# 12. 
Date: 14th Jan 2024<br>
Medium Leetcode: determine-if-two-strings-are-close<br>
Solution: https://leetcode.com/problems/determine-if-two-strings-are-close/submissions/1145505023

In [15]:
class Solution:
    def closeStrings(self, word1: str, word2: str) -> bool:
        n1 = len(word1)
        n2 = len(word2) 

        if n1 != n2:
            return False
        
        else:
            counter1 = collections.Counter(word1)
            counter2 = collections.Counter(word2)
            freq_match = sorted(counter1.values()) == sorted(counter2.values())
            key_match = sorted(counter1.keys()) == sorted(counter2.keys())
            return freq_match and key_match

# Test the solution
assert Solution().closeStrings("abc", "bca") == True
assert Solution().closeStrings("a", "aa") == False
assert Solution().closeStrings("cabbba", "abbccc") == True
assert Solution().closeStrings("faddda", "dccaaa") == False
assert Solution().closeStrings("faddda", "dffaaa") == True
assert Solution().closeStrings("faddda", "dffdaa") == False
assert Solution().closeStrings("aaabbbbccddeeeeefffff", "aaaaabbcccdddeeeeffff") == False

# 13. 
Date: 14th Jan 2024<br>
Medium Leetcode: find-beautiful-indices-in-the-given-array-i<br>
Solution: https://leetcode.com/problems/find-beautiful-indices-in-the-given-array-i/submissions/1145567672/

In [16]:
class Solution:
    def beautifulIndices(self, s: str, a: str, b: str, k: int) -> List[int]:
        result = []
        for i, ch in enumerate(s):
            if ch == a[0] and a == s[i: i + len(a)]:
                if b in s[max(i - k, 0): i + k + len(b)]:
                    result.append(i)
        
        return result

# Test the solution
assert Solution().beautifulIndices("abcd", "a", "b", 1) == [0]
assert Solution().beautifulIndices("abcd", "a", "cd", 1) == []
assert Solution().beautifulIndices("a", "a", "a", 1) == [0]
assert Solution().beautifulIndices("a", "a", "aa", 1) == []
assert Solution().beautifulIndices("vatevavakz", "va", "lbda", 1) == []

# 15. 
Date: 14th Jan 2024<br>
Medium Leetcode: spiral-matrix-ii<br>
Solution: https://leetcode.com/problems/spiral-matrix-ii/submissions/1146146173

In [17]:
class Solution:
    def generateMatrix(self, n: int) -> List[List[int]]: 
        result = [[0 for _ in range(n)] for __ in range(n)]
        directions = (0, 1), (1, 0), (0, -1), (-1, 0)

        curr = 1
        n_layers = math.ceil(n / 2)
        for i in range(n_layers):
            curr_x, curr_y = i, i - 1
            for x_inc, y_inc in directions:
                curr_x, curr_y = curr_x + x_inc, curr_y + y_inc
                while i <= curr_x < n - i and i <= curr_y < n - i:
                    
                    # The below two lines makes me feel like I am cheating
                    if result[curr_x][curr_y] == 0:
                        result[curr_x][curr_y] = curr
                        curr += 1
                        
                    curr_x, curr_y = curr_x + x_inc, curr_y + y_inc
                    
                curr_x, curr_y = curr_x - x_inc, curr_y - y_inc

        return result

# Test the solution
temp = 8
_ = len(str(temp * temp))
for __ in Solution().generateMatrix(temp):
    for ___ in __:
        print (str(___).zfill(_), end=' ')
    print()

01 02 03 04 05 06 07 08 
28 29 30 31 32 33 34 09 
27 48 49 50 51 52 35 10 
26 47 60 61 62 53 36 11 
25 46 59 64 63 54 37 12 
24 45 58 57 56 55 38 13 
23 44 43 42 41 40 39 14 
22 21 20 19 18 17 16 15 


# 16. 
Date: 15th Jan 2024<br>
Medium Leetcode: find-players-with-zero-or-one-losses<br>
Solution: https://leetcode.com/problems/find-players-with-zero-or-one-losses/submissions/1146505590

In [18]:
class Solution:
    def findWinners(self, matches: List[List[int]]) -> List[List[int]]:
        players = set()
        loss_count = dict()
        for winner, loser in matches:
            players.add(winner)
            players.add(loser)
            loss_count[loser] = loss_count.get(loser, 0) + 1

        result = [[],[]]
        for player in sorted(players):
            if player not in loss_count:
                result[0].append(player)
            elif loss_count[player] == 1:
                result[1].append(player)

        return result

# Testing the solution
assert Solution().findWinners([[1,3],[2,3],[3,6],[5,6],[5,7],[4,5],[4,8],[4,9],[10,4],[10,9]]) == [[1,2,10],[4,5,7,8]]
assert Solution().findWinners([[2,3],[1,3],[5,4],[6,4]]) == [[1,2,5,6],[]]

# 17. 
Date: 15th Jan 2024<br>
Medium Leetcode: swap-nodes-in-pairs<br>
Solution: https://leetcode.com/problems/swap-nodes-in-pairs/submissions/1146525714/

In [19]:
class Solution:
    def swapPairs(self, head: ListNode) -> ListNode:

        if not head or not head.next:
            return head

        prev_, curr_, next_ = None, head, head.next
        while next_: 

            curr_.next = next_.next
            next_.next = curr_

            if prev_:
                prev_.next = next_
            else:
                head = next_

            if curr_.next:
                prev_, curr_, next_ = curr_, curr_.next, curr_.next.next
            else:
                break
        
        return head

# Test the solution
assert Solution().swapPairs(ListNode.to_singly_linked_list([])) is None
assert Solution().swapPairs(ListNode.to_singly_linked_list([1])).to_list() == [1]
assert Solution().swapPairs(ListNode.to_singly_linked_list([1, 2])).to_list() == [2, 1]
assert Solution().swapPairs(ListNode.to_singly_linked_list([1, 2, 3])).to_list() == [2, 1, 3]
assert Solution().swapPairs(ListNode.to_singly_linked_list([1, 2, 3, 4])).to_list() == [2, 1, 4, 3]
assert Solution().swapPairs(ListNode.to_singly_linked_list([1, 2, 3, 4, 5])).to_list() == [2, 1, 4, 3, 5]

# 18. 
Date: 15th Jan 2024<br>
Medium Leetcode: simplify-path<br>
Solution: https://leetcode.com/problems/simplify-path/submissions/1146606869

In [20]:
class Solution:
    def simplifyPath(self, path: str) -> str:
        result = []

        # Treat "$" as end of path character
        for ch in path + "$":

            prev = result[-1] if len(result) else None

            # First char, insert as is
            if prev is None:
                result.append(ch)

            # Path related
            elif ch == "/":
                # Do nothing - "//"
                if prev == "/":
                    pass
                # Remove the dot and ignore ch - "./"
                elif prev == ".":
                    result.pop()
                # Remove last four parts - "/..", "/etc/..", "/etc/config/.."; Insert "/"
                elif prev == "..":
                    result = result[:-4]
                    result.append(ch)
                else:
                    result.append(ch)

            # End of path
            elif ch == "$":
                # Ending with a "/", remove it
                if prev == "/" and len(result) > 1:
                    result.pop()
                # Ends with a ".", remove it - "/abc/."
                elif prev == ".":
                    result = result[:-2]
                # Ends with a "..", remove it, go back by 2 dirs - "/abc/.."
                elif prev == "..":
                    result = result[:-4]

                # Say we had "/." or "/..", it would end up as ""
                if not len(result):
                    result.append("/")

            # Everything else is treated as part of dir / file name
            else:
                if prev == "/":
                    result.append(ch)
                else:
                    result[-1] = prev + ch

        return "".join(result)

# Test the solution
assert Solution().simplifyPath("/") == "/"
assert Solution().simplifyPath("/./../") == "/"
assert Solution().simplifyPath("/.gitignore") == "/.gitignore"
assert Solution().simplifyPath("/abc") == "/abc"
assert Solution().simplifyPath("/home//foo/") == "/home/foo"
assert Solution().simplifyPath("/home/kael/././configs/../.vimrc") == "/home/kael/.vimrc"
assert Solution().simplifyPath("/../../../etc////////abc...abc") == "/etc/abc...abc"
assert Solution().simplifyPath("///////......./.../..") == "/......."

# 19. 
Date: 15th Jan 2024<br>
Medium Leetcode: reorder-list<br>
Solution: https://leetcode.com/problems/reorder-list/submissions/1147036556/

In [21]:
class Solution:
    def reorderList(self, head: ListNode) -> ListNode:
        # Find the middle
        slow = fast = head
        while fast and fast.next:
            fast = fast.next.next
            slow = slow.next

        # slow is the start of second half, cut link from first half
        second_start = slow.next
        slow.next = None

        # Reverse list starting from second_start and get the head
        prev, curr = None, second_start
        while curr:
            next_ = curr.next
            curr.next = prev
            prev, curr = curr, next_

        # prev is the start of the reversed second half
        p1, p2 = head, prev
        while p1 and p2:
            next_p1, next_p2 = p1.next, p2.next
            p1.next, p2.next = p2, p1.next
            p1, p2 = next_p1, next_p2

        return head

# Test the solution
assert Solution().reorderList(ListNode.to_singly_linked_list([1])).to_list() == [1]
assert Solution().reorderList(ListNode.to_singly_linked_list([1,2])).to_list() == [1,2]
assert Solution().reorderList(ListNode.to_singly_linked_list([1,2,3])).to_list() == [1,3,2]
assert Solution().reorderList(ListNode.to_singly_linked_list([1,2,3,4])).to_list() == [1,4,2,3]
assert Solution().reorderList(ListNode.to_singly_linked_list([1,2,3,4,5])).to_list() == [1,5,2,4,3]

# 20. 
Date: 16th Jan 2024<br>
Medium Leetcode: insert-delete-getrandom-o1<br>
Solution: https://leetcode.com/problems/insert-delete-getrandom-o1/submissions/1147368883

In [22]:
import random

class RandomizedSet:

    def __init__(self):
        self.length = 0
        self.hashset = set()
        self.randomlist = []
        self.modified = False

    def insert(self, val: int) -> bool:
        'O(1) time complexity'
        if val not in self.hashset:
            self.hashset.add(val)
            self.length += 1
            self.modified = True
            return True
        else:
            return False

    def remove(self, val: int) -> bool:
        'O(1) time complexity'
        if val in self.hashset:
            self.hashset.remove(val)
            self.length -= 1
            self.modified = True
            return True
        else:
            return False
        
    def getRandom(self) -> int:
        'If data was recently modified, takes time to build the list, otherwise O(1) time complexity'
        idx = random.randint(0, self.length - 1) 
        if self.modified:
            self.randomlist = list(self.hashset)
            self.modified = False

        return self.randomlist[idx]

# 21. 
Date: 16th Jan 2024<br>
Medium Leetcode: car-fleet<br>
Solution: https://leetcode.com/problems/car-fleet/submissions/1147391441

In [23]:
class Solution:
    def carFleet(self, target: int, position: List[int], speed: List[int]) -> int:
        # O(n log n)
        position, speed = zip(*sorted(zip(position, speed), key=lambda x: x[0], reverse=True))

        # O(n)
        time = []
        for p, s in zip(position, speed):
            if s != 0:
                time.append((target - p) / s)
            if s == 0:
                break

        # O(n)
        fleet_count = 0
        prev = -math.inf
        for t in time:
            if t > prev:
                fleet_count += 1
                prev = t

        return fleet_count
        
# Testing the solution
assert Solution().carFleet(12, [10,8,0,5,3], [1,4,1,1,3]) == 3
assert Solution().carFleet(10, [3], [3]) == 1
assert Solution().carFleet(100, [0,2,4], [4,2,1]) == 1
assert Solution().carFleet(100, [0,2,4], [4,2,0]) == 0

# 22. 
Date: 16th Jan 2024<br>
Medium Leetcode: k-th-smallest-prime-fraction<br>
Solution: https://leetcode.com/problems/k-th-smallest-prime-fraction/submissions/1147402296/

In [24]:
class Solution:
    def kthSmallestPrimeFraction_(self, arr: List[int], k: int) -> List[int]:
        '''
        This solution only runs as long as neccessary. If k is small this might work better than other solutions
        Could be optimized by going for largest if k is above the mid point of nc2 combinations
        '''
        
        n = len(arr)
        pointers = dict.fromkeys(arr[:-1], 1)
        
        while k > 0:
            minimum_fraction = (math.inf, 1)
            for numerator in pointers:
                denominator = arr[n - pointers[numerator]]
                if (numerator / denominator) <= (minimum_fraction[0] / minimum_fraction[1]):
                    minimum_fraction = numerator, denominator

            # Increment the pointer by 1
            pointers[minimum_fraction[0]] += 1

            if arr[n - pointers[minimum_fraction[0]]] == minimum_fraction[0]:
                pointers.pop(minimum_fraction[0])

            # Decrease k and continue finding the next minimum fraction
            k -= 1

        return minimum_fraction

    def kthSmallestPrimeFraction(self, arr: List[int], k: int) -> List[int]:
        return sorted(itertools.combinations(arr, r=2), key=lambda x: x[0] / x[1])[k - 1]
        
# Testing the solution
assert Solution().kthSmallestPrimeFraction([1, 2, 3, 4, 5], 1) == (1, 5)
assert Solution().kthSmallestPrimeFraction([1, 2, 3, 4, 5], 2) == (1, 4)
assert Solution().kthSmallestPrimeFraction([1, 2, 3, 4, 5], 4) == (2, 5)
assert Solution().kthSmallestPrimeFraction([1, 2, 3, 4, 5], 6) == (2, 4)

# 23. 
Date: 17th Jan 2024<br>
Medium Leetcode: knight-dialer<br>
Solution: https://leetcode.com/problems/knight-dialer/submissions/1148348130

In [25]:
class Solution:
    def knightDialer(self, n: int) -> int:
        """
         1  2  3  4  5  6  7  8  9  0
        [1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
        [2, 2, 2, 3, 0, 3, 2, 2, 2, 2]
        [5, 4, 5, 6, 0, 6, 5, 4, 5, 6]
        """

        MOD = (10 ** 9) + 7
        dialpad_paths = {
            1: [6, 8],
            2: [7, 9],
            3: [4, 8],
            4: [0, 3, 9],
            5: [],
            6: [0, 1, 7],
            7: [2, 6],
            8: [1, 3],
            9: [2, 4],
            0: [4, 6]
        }            

        dp = [1] * 10
        while n > 1:
            new_row = []
            for i in range(10):
                new_row.append(sum(dp[j] for j in dialpad_paths[i]))
                
            dp = new_row
            n -= 1

        return sum(dp) % MOD

# Testing the solution
assert Solution().knightDialer(1) == 10
assert Solution().knightDialer(2) == 20
assert Solution().knightDialer(3131) == 136006598
assert Solution().knightDialer(100) == 540641702
assert Solution().knightDialer(99) == 168850940

# 24. 
Date: 18th Jan 2024<br>
Medium Leetcode: invalid-transactions<br>
Solution: https://leetcode.com/problems/invalid-transactions/submissions/1149291531/

In [26]:
class Solution:
    def invalidTransactions(self, transactions: List[str]) -> List[str]:
        bank = dict()
        for idx, transaction in enumerate(transactions):
            name, time, amount, city = transaction.split(",")
            time = int(time)
            amount = int(amount)
            history = bank.get(name, [])
            history.append((time, amount, city, idx))
            bank[name] = history

        invalid = set()
        for name in bank:
            account = bank[name]
            account.sort()

            for i, (time, amount, city, transaction_idx) in enumerate(account):
                if amount > 1000:
                    invalid.add(transaction_idx)


                for shift in [-1, 1]:
                    while i + shift >= 0 and i + shift < len(account):
                        prev_time, prev_amount, prev_city, prev_tran_idx = account[i + shift]
                        if abs(prev_time - time) <= 60:
                            if prev_city != city:
                                invalid.add(prev_tran_idx)
                                invalid.add(transaction_idx)
                                
                            if shift < 0:
                                shift -= 1
                            else:
                                shift += 1
                                
                        else:
                            break

        return [transactions[_] for _ in invalid]

# Testing the solution
temp = [
    [
        ["alice,20,800,mtv","alice,50,100,beijing"],
        ["alice,50,100,beijing","alice,20,800,mtv"]
    ],
    
    [
        ["alice,20,800,mtv","alice,50,1200,mtv"],
        ["alice,50,1200,mtv"]
    ],
    
    [
        ["alice,20,800,mtv","bob,50,1200,mtv"],
        ["bob,50,1200,mtv"]
    ],
    
    [
        ["alice,20,800,mtv","bob,50,1200,mtv","alice,20,800,mtv","alice,50,1200,mtv","alice,20,800,mtv","alice,50,100,beijing"],
        ["alice,20,800,mtv","bob,50,1200,mtv","alice,20,800,mtv","alice,50,1200,mtv","alice,20,800,mtv","alice,50,100,beijing"]
    ],

    [
        ["alex,676,260,bangkok","bob,656,1366,bangkok","alex,393,616,bangkok","bob,820,990,amsterdam","alex,596,1390,amsterdam"],  
        ["bob,656,1366,bangkok","alex,596,1390,amsterdam"]
    ],

    [
        ["alice,20,1220,mtv","alice,20,1220,mtv"],  
        ["alice,20,1220,mtv","alice,20,1220,mtv"]
    ],

    [
        ["alice,20,800,mtv","alice,50,100,mtv","alice,51,100,frankfurt"],
        ["alice,20,800,mtv","alice,50,100,mtv","alice,51,100,frankfurt"]
    ],
]

for inp, op in temp:
    assert sorted(Solution().invalidTransactions(inp)) == sorted(op), f"failed for {inp}"

# 25. 
Date: 18th Jan 2024<br>
Medium Leetcode: longest-arithmetic-subsequence<br>
Solution: https://leetcode.com/problems/longest-arithmetic-subsequence/submissions/1149314640

In [27]:
class Solution:
    def longestArithSeqLength_(self, nums: List[int]) -> int:
        """
        For size 2, return 2 (nums[0], nums[1] regardless of what the nums are)
        """

        @functools.cache
        def backtrack(curr_idx: int, diff: int) -> int:
            curr = nums[curr_idx]
            if curr + diff in idx_dict:
                for next_idx in idx_dict[curr + diff]:
                    if next_idx > curr_idx:
                        # We only need to backtrack into the first index > curr_idx
                        return 1 + backtrack(next_idx, diff)
                else:
                    return 0
            else:
                return 0

        N = len(nums)

        idx_dict = dict()
        for i, n in enumerate(nums):
            indices = idx_dict.get(n, [])
            indices.append(i)
            idx_dict[n] = indices

        if N == 2:
            return 2
        else:
            maxlen = 2
            for i in range(N - 1):
                for j in range(i + 1, N):
                    maxlen = max(maxlen, 2 + backtrack(j, nums[j] - nums[i]))

            return maxlen

    def longestArithSeqLength(self, A):
        # https://leetcode.com/problems/longest-arithmetic-subsequence/solutions/274611/java-c-python-dp/
        # Intution is kind of same as what I had done above?
        dp = {}
        for i in range(len(A) - 1):
            for j in range(i + 1, len(A)):
                dp[j, A[j] - A[i]] = dp.get((i, A[j] - A[i]), 1) + 1

        return max(dp.values())

# Testing the solution
assert Solution().longestArithSeqLength([3,6,9,12]) == 4
assert Solution().longestArithSeqLength([3,6,9,6,12]) == 4
assert Solution().longestArithSeqLength([9,4,7,2,10]) == 3
assert Solution().longestArithSeqLength([20,1,15,3,10,5,8]) == 4
assert Solution().longestArithSeqLength([1,2]) == 2
assert Solution().longestArithSeqLength([1,2,2,3,3,3,3,3,3,3,4,4,5,6,7,8,8,8,8,8,8,8,8,8]) == 9

# 26. 
Date: 18th Jan 2024<br>
Medium Leetcode: longest-palindromic-substring<br>
Solution: https://leetcode.com/problems/longest-palindromic-substring/submissions/1149814283

In [28]:
class Solution:
    def longestPalindrome(self, s: str) -> str:
        '''
        "abaaba" -  2
        "abaaba" -  3
        "abaaaba" - 3
        '''

        N = len(s)

        def longestPalindromeAtIndex(idx: int) -> str:

            # Store the longest
            longest = ""

            # Assuming i is the mid point where palindrome is odd char long
            # Assuming i is the left mid point where palindrome is even char long
            # Assuming i is the right mid point where palindrome is even char long
            for left, right in [[idx - 1, idx + 1], [idx, idx + 1], [idx - 1, idx]]:

                while left >= 0 and right < N and s[left] == s[right]:
                    left -= 1
                    right += 1

                longest = longest if right - left - 2 < len(longest) else s[left + 1: right]

            return longest

        longest = ""
        for i in range(N):
            curr = longestPalindromeAtIndex(i)
            if len(curr) > len(longest):
                longest = curr

        return longest

# Testing the Solution
assert Solution().longestPalindrome("abaaba") == "abaaba"
assert Solution().longestPalindrome("abaaba") == "abaaba"
assert Solution().longestPalindrome("abaaaba") == "abaaaba"
assert Solution().longestPalindrome("babad") == "bab"
assert Solution().longestPalindrome("cbbd") == "bb"

# 27. 
Date: 18th Jan 2024<br>
Medium Leetcode: gas-station<br>
Solution: https://leetcode.com/problems/gas-station/submissions/1149948924

In [29]:
class Solution:
    def canCompleteCircuit(self, gas: List[int], cost: List[int]) -> int:
        """
        [1,2,3,4,5]
        [3,4,5,1,2]

        [-2, -2, -2, 3, 3]
        To complete a circuit, sum should be >= 0
        """ 

        N = len(gas)
        dp = [gas[i] - cost[i] for i in range(N)]

        if sum(dp) < 0:
            return -1

        else:

            total = 0
            for i in range(N - 1, -1, -1):
                dp[i] += total
                total = dp[i]

            return dp.index(max(dp))

# Testing the solution
assert Solution().canCompleteCircuit([1,2,3,4,5], [3,4,5,1,2]) == 3
assert Solution().canCompleteCircuit([2,3,4], [3,4,3]) == -1

# 28. 
Date: 19th Jan 2024<br>
Medium Leetcode: minimum-falling-path-sum<br>
Solution: https://leetcode.com/problems/minimum-falling-path-sum/submissions/1150236802

In [30]:
class Solution:
    def minFallingPathSum(self, matrix: List[List[int]]) -> int:
        N = len(matrix)
        dp = matrix[N - 1]
        for i in range(N - 2, -1, -1):
            newrow = []
            for j in range(N):
                newrow.append(matrix[i][j] + min(dp[j], dp[j - 1] if j - 1 >= 0 else math.inf, dp[j + 1] if j + 1 < N else math.inf))
            dp = newrow

        return min(dp)

# Testing the solution
assert Solution().minFallingPathSum([[2,1,3],[6,5,4],[7,8,9]]) == 13
assert Solution().minFallingPathSum([[-19,57],[-40,-5]]) == -59
assert Solution().minFallingPathSum([[1]]) == 1

# 29. 
Date: 19th Jan 2024<br>
Medium Leetcode: decode-string<br>
Solution: https://leetcode.com/problems/decode-string/submissions/1150250564

In [31]:
class Solution:
    def decodeString(self, s: str) -> str:
        stack = []
        for ch in s:
            if ch != "]":
                stack.append(ch)
            else:
                
                to_repeat = []
                prev_ch = ""
                while prev_ch != "[":
                    to_repeat.append(prev_ch)
                    prev_ch = stack.pop()
                to_repeat = "".join(to_repeat[::-1])

                repeat_count = []
                while len(stack):
                    prev_ch = stack[-1]
                    if prev_ch.isnumeric():
                        repeat_count.append(prev_ch)
                        stack.pop()
                    else:
                        break
                repeat_count = int("".join(repeat_count[::-1]))

                stack.append(to_repeat * repeat_count)

        return "".join(stack)

# Testing the solution
assert Solution().decodeString("1[y]5[z2[x]3[k2[p]]]") == "yzxxkppkppkppzxxkppkppkppzxxkppkppkppzxxkppkppkppzxxkppkppkpp"

# 30. 
Date: 19th Jan 2024<br>
Medium Leetcode: clone-graph<br>
Solution: https://leetcode.com/problems/clone-graph/submissions/1150260315/

In [32]:
class Node:
    def __init__(self, val = 0, neighbors = None):
        self.val = val
        self.neighbors = neighbors if neighbors is not None else []

class Solution:
    def __init__(self):
        self.graph = dict()

    def cloneGraph(self, node: Node) -> Node:
        if not node:
            return node
        elif node.val in self.graph:
            return self.graph[node.val]
        else:
            clone = Node(node.val, [])
            self.graph[node.val] = clone
            neighbors = [self.cloneGraph(neighbor) for neighbor in node.neighbors]
            clone.neighbors = neighbors
            return clone

# 31. 
Date: 19th Jan 2024<br>
Medium Leetcode: count-good-nodes-in-binary-tree<br>
Solution: https://leetcode.com/problems/count-good-nodes-in-binary-tree/submissions/1150270252

In [33]:
class Solution:
    def __init__(self):
        self.goodNodeCount = 0

    def goodNodes(self, curr: BinaryTreeNode, maximum=-math.inf) -> int:

        if not curr:
            return

        if curr.val >= maximum:
            self.goodNodeCount += 1

        self.goodNodes(curr.left, maximum=max(maximum, curr.val)) 
        self.goodNodes(curr.right, maximum=max(maximum, curr.val)) 

        return self.goodNodeCount

# Testing the solution
assert Solution().goodNodes(BinaryTreeNode.from_array([3,1,4,3,None,1,5])) == 4
assert Solution().goodNodes(BinaryTreeNode.from_array([3,1,5,3,None,1,4])) == 3
assert Solution().goodNodes(BinaryTreeNode.from_array([1])) == 1
assert Solution().goodNodes(BinaryTreeNode.from_array([3,3,None,4,2])) == 3

# 32. 
Date: 19th Jan 2024<br>
Medium Leetcode: redundant-connection<br>
Solution: https://leetcode.com/problems/redundant-connection/submissions/1150754530

In [34]:
class Solution:
    def findRedundantConnection_(self, edges: List[List[int]]) -> List[int]:
        def DFS(start, end, visited=set()) -> bool:
            neighbours = paths[start]
            if end in neighbours:
                return True
            else:
                return any(DFS(curr, end, {*visited, start}) for curr in neighbours if curr not in visited)

        paths = dict()
        for start, end in edges:
            if start not in paths or end not in paths:
                paths[start] = paths.get(start, set())
                paths[end] = paths.get(end, set())
                paths[start].add(end)
                paths[end].add(start)
            else:
                # DFS to see if a connection already exists, create shortcuts
                if DFS(start, end):
                    return [start, end]
                else:
                    paths[start].add(end)
                    paths[end].add(start)

    def findRedundantConnection(self, edges: List[List[int]]) -> List[int]:
        islands = []
        for start, end in edges:

            # Find the island where start, end can be found
            start_idx = end_idx = -1
            for i in range(len(islands)):
                if start in islands[i]:
                    start_idx = i
                if end in islands[i]:
                    end_idx = i
                if start_idx != -1 and end_idx != -1:
                    break

            # Based on whether start, end exists and where, we execute different scenarios
            if start_idx == -1 and end_idx == -1:
                islands.append({start, end})
            elif start_idx == -1:
                islands[end_idx].add(start)
            elif end_idx == -1:
                islands[start_idx].add(end)
            else:
                if start_idx == end_idx:
                    return [start, end]
                else:
                    # Merge the islands
                    island_1 = islands.pop(start_idx)
                    island_2 = islands.pop(end_idx - (1 if start_idx < end_idx else 0))
                    islands.append({*island_1, *island_2})

# Testing the solution
assert Solution().findRedundantConnection([[1,2],[1,3],[2,3]]) == [2, 3]
assert Solution().findRedundantConnection([[1,2],[2,3],[3,4],[1,4],[1,5]]) == [1, 4]
assert Solution().findRedundantConnection([[3,4],[1,2],[2,4],[3,5],[2,5]]) == [2, 5]

# 33. 
Date: 19th Jan 2024<br>
Medium Leetcode: find-peak-element<br>
Solution: https://leetcode.com/problems/find-peak-element/submissions/1150772011

In [35]:
class Solution:
    def findPeakElement(self, nums: List[int]) -> int:

        N = len(nums)
        start, end = 0, N - 1
        while start <= end:

            mid = (start + end) // 2
            curr = nums[mid]
            left = nums[mid - 1] if mid - 1 >= 0 else -math.inf
            right = nums[mid + 1] if mid + 1 < N else -math.inf

            if left < curr > right:
                return mid
            elif left < curr < right:
                start = mid + 1
            else:
                end = mid - 1

        return -1

# Testing the solution
assert Solution().findPeakElement([1,2,1,3,5,6,4]) == 5
assert Solution().findPeakElement([1,2,3,1]) == 2
assert Solution().findPeakElement([0,1,2,3]) == 3
assert Solution().findPeakElement([3,2,1,0]) == 0
assert Solution().findPeakElement([1,-5,-2,-3]) == 0

# 34. 
Date: 20th Jan 2024<br>
Medium Leetcode: sum-of-subarray-minimums<br>
Solution: https://leetcode.com/problems/sum-of-subarray-minimums/submissions/1151318247

In [36]:
class Solution:
    def sumSubarrayMins_(self, arr: List[int]) -> int:
        N = len(arr)
        MOD = (10 ** 9) + 7
        result = 0
        for i in range(N):
            minimum = arr[i]
            for j in range(1, N - i + 1):
                minimum = min(minimum, arr[i + j - 1])
                result += minimum

        return result % MOD

    def sumSubarrayMins(self, arr: List[int]) -> int:

        N = len(arr)
        MOD = (10 ** 9) + 7

        # count of continuous Numbers left, right of arr[i] that are greater than itself
        left = [i for i in range(N)]
        stack = []
        for i in range(N - 1, -1, -1):
            while len(stack) and stack[-1][1] > arr[i]:
                idx, _ = stack.pop()
                left[idx] = idx - i - 1
            stack.append((i, arr[i]))

        right = [N - i - 1 for i in range(N)]
        stack = []
        for i in range(N):
            # the '=' sign is needed I think (don't fully understand it) because 
            # we need to include the same value only once
            # Without '=' sign on both, we could count it on both left and right
            # With '=' sign on both, we would fail to count it on both left and right
            while len(stack) and stack[-1][1] >= arr[i]: 
                idx, _ = stack.pop()
                right[idx] = i - idx - 1
            stack.append((i, arr[i]))

        result = 0
        for i in range(N):
            result += (left[i] + 1) * (right[i] + 1) * arr[i]

        return result % MOD

# Testing the solution
assert Solution().sumSubarrayMins([3,1,2,4]) == 17
assert Solution().sumSubarrayMins([11,81,94,43,3]) == 444
assert Solution().sumSubarrayMins([71,55,82,55]) == 593

# 35. 
Date: 20th Jan 2024<br>
Medium Leetcode: subsets-ii<br>
Solution: https://leetcode.com/problems/subsets-ii/submissions/1151410592

In [37]:
class Solution:
    def subsetsWithDup(self, nums: List[int]) -> List[List[int]]:
        result = {tuple()}
        for n in sorted(nums):
            curr = {*result}
            for comb in curr:
                result.add((*comb, n))

        return result

# Testing the solution
Solution().subsetsWithDup([4,4,4,1,4])

{(),
 (1,),
 (1, 4),
 (1, 4, 4),
 (1, 4, 4, 4),
 (1, 4, 4, 4, 4),
 (4,),
 (4, 4),
 (4, 4, 4),
 (4, 4, 4, 4)}

# 36. 
Date: 20th Jan 2024<br>
Medium Leetcode: combinations<br>
Solution: https://leetcode.com/problems/combinations/submissions/1151420623

In [38]:
class Solution:
    def combine(self, n: int, k: int) -> List[List[int]]:
        arr = list(range(1, n + 1))

        result = []
        def backtrack(arr, curr=[]):
            if len(curr) == k:
                result.append(curr)
            elif arr:
                backtrack(arr[1:], [*curr, arr[0]])
                backtrack(arr[1:], curr)

        backtrack(arr, [])
        return result

# Testing the solution
assert sorted(Solution().combine(4, 2)) == [[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [3, 4]]
assert sorted(Solution().combine(4, 3)) == [[1, 2, 3], [1, 2, 4], [1, 3, 4], [2, 3, 4]]

# 37. 
Date: 20th Jan 2024<br>
Medium Leetcode: reverse-words-in-a-string<br>
Solution: https://leetcode.com/problems/reverse-words-in-a-string/submissions/1151520485

In [39]:
class Solution:
    def __init__(self):
        self.words = []

    def reverseWords_(self, s: str) -> str:
        word = []
        for ch in s + " ":
            if not ch.isspace():
                word.append(ch)
            elif word:
                self.words.append("".join(word))
                word = []

        return " ".join(self.words[::-1])
    
    def reverseWords(self, s: str) -> str:
        
        def reverseWord(s_: List[str], start_: int, end_: int): 
            while start_ < end_:
                s_[start_], s_[end_] = s_[end_], s_[start_]
                start_ += 1
                end_ -= 1

        # O(1) space compelexity solution
        sentence = list(s)
        N = len(s)

        # Remove white spaces in place
        prev = " "
        i = j = 0
        while j < N:
            if sentence[j].isspace() and prev.isspace():
                j += 1
            else:
                sentence[i] = sentence[j]
                prev = sentence[j]
                i += 1
                j += 1

        i = i if not sentence[i - 1].isspace() else i - 1
        sentence = sentence[:i]
        N = max(i, 0)

        # Reverse the sentence as a whole
        sentence = sentence[::-1]

        # Reverse the words in the sentence
        start = 0
        for i in range(N + 1):
            if i == N or sentence[i].isspace():
                reverseWord(sentence, start, i - 1)
                start = i + 1

        return "".join(sentence)

# Testing the solution
for temp in [
    "     hello   world   is a     coooool programming    phrase  ",
    "a asd a b c d"
]:
    assert Solution().reverseWords(temp) == Solution().reverseWords_(temp)

# 38. 
Date: 20th Jan 2024<br>
Medium Leetcode: angle-between-hands-of-a-clock<br>
Solution: https://leetcode.com/problems/angle-between-hands-of-a-clock/submissions/1151539829

In [40]:
class Solution:
    def angleClock(self, hour: int, minutes: int) -> float:
        minutes_deg = (minutes / 60) * 360
        hours_deg = ((hour / 12) * 360) + ((minutes / 60) * 30)
        hours_deg %= 360
        diff = abs(hours_deg - minutes_deg)
        return min(diff, 360 - diff)

# Testing the solution
assert Solution().angleClock(3, 15) == 7.5
assert Solution().angleClock(12, 59) == 35.5
assert Solution().angleClock(12, 30) == 165

# 39. 
Date: 21th Jan 2024<br>
Medium Leetcode: house-robber<br>
Solution: https://leetcode.com/problems/house-robber/submissions/1152099761

In [41]:
class Solution:

    def rob_(self, nums: List[int], i: int = 0) -> int:
        'Recursively solve the dp, this will work even if there are -ve values'
        
        @functools.cache
        def backtrack(i: int = 0) -> int:
            if i >= len(nums):
                return 0

            else:
                return max(
                    nums[i] + backtrack(i + 2),
                    backtrack(i + 1)
                )
            
        return backtrack()
        
    def rob(self, nums: List[int], i: int = 0) -> int:
        'Itertively solve the problem'
        N = len(nums)
        dp = nums + [0, 0]

        for i in range(N - 1, -1, -1):
            dp[i] = max(dp[i] + dp[i + 2], dp[i + 1])

        return dp[0]

# Testing the solution
assert Solution().rob_([1, 2, -100, -50, 100]) == Solution().rob_([1, 2, -100, -50, 100])
assert Solution().rob_([1, 2, -3, -5, 0, 1]) == Solution().rob_([1, 2, -3, -5, 0, 1])
assert Solution().rob_([1, 2, 1]) == Solution().rob_([1, 2, 1])

# 40. 
Date: 21th Jan 2024<br>
Medium Leetcode: minimum-number-of-pushes-to-type-word-ii<br>
Solution: https://leetcode.com/problems/minimum-number-of-pushes-to-type-word-ii/submissions/1152173013

In [42]:
class Solution:
    def minimumPushes(self, word: str) -> int:
        counter = collections.Counter(word)
        n_pushes = dict()
        key_count = 0
        for k, v in counter.most_common():
            n_pushes[k] = (key_count // 8) + 1
            key_count += 1
        
        count = 0
        for k in word:
            count += n_pushes[k]
        return count

# Testing the solution
assert Solution().minimumPushes("aabbccddeeffgghhiiiiii") == 24
assert Solution().minimumPushes("xyzxyzxyzxyz") == 12
assert Solution().minimumPushes("abcde") == 5

# 41. 
Date: 21th Jan 2024<br>
Medium Leetcode: count-the-number-of-houses-at-a-certain-distance-i<br>
Solution: https://leetcode.com/problems/count-the-number-of-houses-at-a-certain-distance-i/submissions/1152316123/

In [43]:
class Solution:
    def countOfPairs(self, n: int, x: int, y: int) -> List[int]:

        # Adjacency list creation
        paths = dict()
        for i in range(1, n):
            paths[i] = {*paths.get(i, set()), i + 1}
            paths[i + 1] = {*paths.get(i + 1, set()), i}
        
        # Adding the shortcut
        if x != y:
            paths[x].add(y)
            paths[y].add(x)

        result = [0] * n
        for curr in range(1, n + 1):
            visited = {curr}
            to_visit = paths[curr]
            dist = 1
            while to_visit:
                next_to_visit = set()
                for next_ in to_visit:
                    if next_ not in visited:
                        result[dist - 1] += 1
                        next_to_visit = {*next_to_visit, *paths[next_]}
                        visited.add(next_)
                to_visit = next_to_visit
                dist += 1
        
        return result

# Testing the solution
assert Solution().countOfPairs(3, 1, 3) == [6, 0, 0]
assert Solution().countOfPairs(5, 2, 4) == [10,8,2,0,0]
assert Solution().countOfPairs(4, 2, 1) == [6,4,2,0]

# 42. 
Date: 21th Jan 2024<br>
Medium Leetcode: basic-calculator-ii<br>
Solution: https://leetcode.com/problems/basic-calculator-ii/submissions/1152355111

In [44]:
class Solution:
    def calculate(self, s: str) -> int:
        'DMAS - Division, Multiplication, Addition, Subtraction'

        # Store the results here
        result = []

        # Remove whitespaces and aggregate the integers, ops together
        acc = []
        for ch in s + "$":
            if ch.isnumeric():
                acc.append(ch)
            elif ch in ("+", "/", "*", "-"):
                result.append(int("".join(acc)))
                result.append(ch)
                acc = []
            elif ch == "$":
                result.append(int("".join(acc)))

        # Multiplication / Division first followed by addition / subtraction in LTR order
        for op in ["*/", "+-"]:
            acc = []
            prev = "$" # Some random character
            for ch in result:
                if type(ch) == int:
                    if prev in op:
                        if prev == "/":
                            acc.append(acc.pop() // ch)
                        elif prev == "*":
                            acc.append(acc.pop() * ch)
                        elif prev == "+":
                            acc.append(acc.pop() + ch)
                        else:
                            acc.append(acc.pop() - ch)
                    else:
                        acc.append(ch)
                elif ch not in op:
                    acc.append(ch)
                prev = ch

            result = acc

        assert len(result) == 1
        return result[0]

# Testing the solution
assert Solution().calculate("1+2*5/3+6/4*2") == 6
assert Solution().calculate("  44/2-2*    3+ 5") == 21
assert Solution().calculate(" 4 / 2 / 2") == 1
assert Solution().calculate("3+2*2") == 7
assert Solution().calculate(" 3/2 ") == 1
assert Solution().calculate(" 3+5 / 2 ") == 5
assert Solution().calculate("4/2-2*3+5") == 1
assert Solution().calculate("2 - 6 + 5") == 1

# 43. 
Date: 22th Jan 2024<br>
Medium Leetcode: 4sum<br>
Solution: https://leetcode.com/problems/4sum/submissions/1153131773

In [45]:
class Solution:
    def fourSum_(self, nums: List[int], target: int) -> List[List[int]]:
        "Time complexity of O(N^3). It always runs O(N^3) iterations"

        N = len(nums)
        result = set()

        indices = dict()
        for i in range(N):
            index = indices.get(nums[i], set())
            index.add(i)
            indices[nums[i]] = index 

        for i in range(N - 3):
            for j in range(i + 1, N - 2):
                for k in range(j + 1, N - 1):
                    total = nums[i] + nums[j] + nums[k]
                    index = set.difference(indices.get(target - total, set()), {i, j, k})
                    if len(index):
                        l = index.pop()
                        result.add(tuple(sorted((nums[i], nums[j], nums[k], nums[l]))))

        return result

    def fourSum(self, nums: List[int], target: int) -> List[List[int]]:
        "Time compelexity is O(N^3) and runs the entire iteration"

        # Sort the array for using the two pointer approach
        nums.sort()
        N = len(nums)

        result = set()
        for i in range(N - 3):
            for j in range(i + 1, N - 2):
                remaining = target - nums[i] - nums[j]
                start, end = j + 1, N - 1
                while start < end:
                    total = nums[start] + nums[end]
                    if total < remaining:
                        start += 1
                    elif total > remaining:
                        end -= 1
                    else:
                        result.add(tuple(sorted((nums[i], nums[j], nums[start], nums[end]))))
                        start += 1
                        end -= 1

        return result

# Testing the solution
assert sorted(Solution().fourSum([2,2,2,2,2], 8)) == [(2,2,2,2)]
assert sorted(Solution().fourSum([-1,-2,1,0,1,2,3,-3,0,1,2], -6)) == [(-3,-2,-1,0)]
assert sorted(Solution().fourSum([1,2,2,2], 0)) == []
assert sorted(Solution().fourSum([1,2,2,2], 7)) == [(1,2,2,2)]
assert sorted(Solution().fourSum([-3,-1,0,2,4,5], 0)) == [(-3,-1,0,4)]
assert sorted(Solution().fourSum([0,0,0], 0)) == []

# 44. 
Date: 23th Jan 2024<br>
Medium Leetcode: maximum-length-of-a-concatenated-string-with-unique-characters<br>
Solution: https://leetcode.com/problems/maximum-length-of-a-concatenated-string-with-unique-characters/submissions/1154144877

In [46]:
class Solution:
    def maxLength(self, arr: List[str]) -> int:

        def backtrack(curr: int, acc: set):
            if len(setArr) == 0 or curr >= N:
                nonlocal maxLen
                maxLen = max(maxLen, len(acc))
            else:
                if not set.intersection(setArr[curr], acc):
                    backtrack(curr + 1, {*acc, *setArr[curr]})
                backtrack(curr + 1, acc)

        setArr = list(map(set, arr))
        N = len(arr)
        maxLen = -math.inf

        backtrack(0, set())
        return maxLen

# Testing the solution
assert Solution().maxLength(["un","iq","ue"]) == 4
assert Solution().maxLength(["cha","r","act","ers"]) == 6
assert Solution().maxLength(["abcdefghijklmnopqrstuvwxyz"]) == 26

# 45. 
Date: 23th Jan 2024<br>
Medium Leetcode: group-the-people-given-the-group-size-they-belong-to<br>
Solution: https://leetcode.com/problems/group-the-people-given-the-group-size-they-belong-to/submissions/1154160920/

In [47]:
class Solution:
    def groupThePeople(self, groupSizes: List[int]) -> List[List[int]]:

        N = len(groupSizes) 
        result = []
        store = dict()
        for i in range(N):
            groupSize = groupSizes[i]
            curr = store.get(groupSize, [])

            curr.append(i)
            if len(curr) == groupSize:
                result.append(curr[:])
                curr = []

            store[groupSize] = curr

        return result

# Testing the solution
assert Solution().groupThePeople([3,3,3,3,3,1,3]) == [[0,1,2],[5],[3,4,6]]
assert Solution().groupThePeople([2,1,3,3,3,2]) == [[1],[2,3,4],[0,5]]
assert Solution().groupThePeople([1,1,1,1,1,1,1]) == [[0],[1],[2],[3],[4],[5],[6]]
assert Solution().groupThePeople([5,5,5,5,5]) == [[0,1,2,3,4]]
assert Solution().groupThePeople([1,2,2,3,3,3,4,4,4,4]) == [[0],[1,2],[3,4,5],[6,7,8,9]]

# 46. 
Date: 24th Jan 2024<br>
Medium Leetcode: pseudo-palindromic-paths-in-a-binary-tree<br>
Solution: 

In [48]:
class Solution:
    def pseudoPalindromicPaths (self, root: BinaryTreeNode) -> int: 
        
        """
        Count the digits that are occuring 1 times. It should 1 or 0.
        11221 - palindrome (21112)
        1122 - palindrome (1221)

        We can alternatively use an array and keep track of the count of times each digit comes up
        In the end repeat, but bit solution is more memory efficient.
        """

        def backtrack(root: BinaryTreeNode, count: int = 0):
            if not root:
                return 0
            elif not root.left and not root.right:
                count ^= (1 << (root.val - 1)) 
                return int(count & (count - 1) == 0)
            else:
                count ^= (1 << (root.val - 1)) 
                return backtrack(root.left, count) + backtrack(root.right, count)

        return backtrack(root)

# Testing the solution
assert Solution().pseudoPalindromicPaths(BinaryTreeNode.from_array([2,1,1,1,3,None,None,None,None,None,1])) == 1
assert Solution().pseudoPalindromicPaths(BinaryTreeNode.from_array([2,3,1,3,1,None,1])) == 2
assert Solution().pseudoPalindromicPaths(BinaryTreeNode.from_array([1])) == 1

# 47. 
Date: 24th Jan 2024<br>
Medium Leetcode: minimum-time-difference<br>
Solution: https://leetcode.com/problems/minimum-time-difference/submissions/1155129074/

In [49]:
class Solution:
    def findMinDifference(self, timePoints: List[str]) -> int:
        def timestamp(s: str) -> int:
            h, m = map(int, s.split(":"))
            return (h * 60) + m
            
        ts = sorted(map(timestamp, timePoints))
        N, minDiff = len(ts), math.inf
        for i in range(N):
            diff = ts[i] - ts[i - 1]
            diff = diff if diff >= 0 else 1440 + diff
            minDiff = min(minDiff, diff)

        return minDiff

# Testing the solution
assert Solution().findMinDifference(["23:59","00:00"]) == 1
assert Solution().findMinDifference(["00:00","23:59","00:00"]) == 0
assert Solution().findMinDifference(["22:00","22:59","23:05", "23:45", "00:00"]) == 6
assert Solution().findMinDifference(["00:00", "01:30", "02:00", "02:40", "03:30", "06:00", "07:30", "15:00", "17:20", "19:50", "22:30"]) == 30
assert Solution().findMinDifference(["00:00", "23:50", "23:10", "23:02", "22:58", "22:01", "21:59"]) == 2